# Representational similarity analysis



https://arxiv.org/pdf/1905.06401

present RSA as a vari-
ant of pattern-information analysis, to be applied
for understanding neural activation patterns in hu-
man brains, for example syntactic computations
(Tyler et al., 2013) or sensory cortical process-
ing (Yamins and DiCarlo, 2016). The core idea
is to find connections between data from neu-
roimaging, behavioral experiments and computa-
tional modeling by correlating representations of
stimuli in each of these representation spaces via
their pairwise (dis)similarities. RSA has also been
used for measuring similarities between neural-
network representation spaces (

In [ ]:
# Code taken from: https://github.com/gchrupala/ursa/blob/master/ursa/regress.py

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, r2_score, make_scorer


class Regress:
    default_alphas = [ 10**n for n in range(-3, 2) ]
    metrics = dict(mse       = make_scorer(mean_squared_error, greater_is_better=False),
                   r2        = make_scorer(r2_score, greater_is_better=True),
                   pearson_r = make_scorer(pearson_r_score, greater_is_better=True))


    def __init__(self, cv=10, alphas=default_alphas):
        self.cv = cv
        self.grid =  {'alpha': alphas }
        self._model = GridSearchCV(Ridge(), self.grid, scoring=self.metrics, cv=self.cv, return_train_score=False, refit=False)

    def fit(self, X, Y):
        self._model.fit(X, Y)
        result = { name: {} for name in self.metrics.keys() }
        for name, scorer in self.metrics.items():
            mean = self._model.cv_results_["mean_test_{}".format(name)]
            std  = self._model.cv_results_["std_test_{}".format(name)]
            best = mean.argmax()
            result[name]['mean'] = mean[best] * scorer._sign
            result[name]['std']  = std[best]
            result[name]['alpha'] = self.grid['alpha'][best]
        self._report = result

    def fit_report(self, X, Y):
        self.fit(X, Y)
        return self.report()

    def report(self):
        return self._report

In [ ]:
# Code taken from: https://github.com/gchrupala/correlating-neural-and-symbolic-representations-of-language/blob/master/rsa/correlate.py

import numpy as np
import torch


def rsa(A, B):
    "Returns the correlation between the similarity matrices for A and B."
    M_A = cosine_matrix(A, A)
    M_B = cosine_matrix(B, B)
    return pearson(triu(M_A), triu(M_B), dim=0)


def cosine_matrix(U, V):
    "Returns the matrix of cosine similarity between each row of U and each row of V."
    U_norm = U / U.norm(2, dim=1, keepdim=True)
    V_norm = V / V.norm(2, dim=1, keepdim=True)
    return U_norm @ V_norm.t()


def triu(x):
    "Extracts upper triangular part of a matrix, excluding the diagonal."
    ones = torch.ones_like(x)
    return x[torch.triu(ones, diagonal=1) == 1]


def pearson(x, y, dim=0, eps=1e-8):
    "Returns Pearson's correlation coefficient."
    x1 = x - torch.mean(x, dim)
    x2 = y - torch.mean(y, dim)
    w12 = torch.sum(x1 * x2, dim)
    w1 = torch.norm(x1, 2, dim)
    w2 = torch.norm(x2, 2, dim)
    return w12 / (w1 * w2).clamp(min=eps)


def cor(a, b):
    return pearson(C.triu(a), C.triu(b), dim=0).item()


def rsa_report(data_enc1: dict, data_enc2: dict, cv=10):
    """Compute RSA and RSA_regress scores for two different encoders."""

    # RSA
    D_rep = 1 - cosine_matrix(data_enc["test"], data_enc["test"])
    D_rep2 = 1 - cosine_matrix(data_enc2["test"], data_enc2["test"])
    rsa = cor(D_rep, D_rep2)

    # RSA_regress
    D_rep = (
        1 - C.cosine_matrix(data_enc["test"], data_enc["ref"]).detach().cpu().numpy()
    )
    D_rep2 = (
        1 - C.cosine_matrix(data_enc2["test"], data_enc2["ref"]).detach().cpu().numpy()
    )
    r = Regress(cv=cv)
    rsa_reg = r.fit_report(D_rep, D_rep2)
    return dict(rsa=rsa, rsa_regress=rsa_reg)

Basic RSA measures correlation
between similarities in two different representa-
tions globally, i.e. how close they are in their total-
ity. In contrast, diagnostic models answer a more
specific question: to what extent a particular type
of information can be extracted from a given rep-
resentation. For example, while for a particular
neural encoding of sentences it may be possible to
predict the length of the sentence with high accu-
racy, the RSA between this representation and the
strings represented only by their length may be rel-
atively small in magnitude, since the neural repre-
sentation may be encoding many other aspects of
the input in addition to its length


The scores according to RSA in some cases show a different picture. This is expected, as RSA answers a substantially different question than the other two approaches: it looks at how the whole representations match in their similarity structure, whereas both the diagnostic model and RSAREGRESS focus on the part of the representation that encodes the target information the strongest.

In [ ]:
# Code adapted from: https://github.com/gchrupala/correlating-neural-and-symbolic-representations-of-language/blob/master/rsa/report.py

def run_rsa():
#   try:
#       data_sent = json.load(open("data/out/ewt.json"))
#   except FileNotFoundError:
#       S.ewt_json()
#       data_sent = json.load(open("data/out/ewt.json"))
    try:
        data = torch.load("data/out/ewt_embed.pt")
    except FileNotFoundError:
        S.ewt_embed()
        data = torch.load("data/out/ewt_embed.pt")

    result = {}

    result[alpha] = dict(bow=dict(), bert=dict(), bert24=dict(), infersent=dict())

    data_enc_bow = dict(test=data['bow']['test'], ref=data['bow']['ref'])
    result[alpha]['bow']=RSA_report(data_tk, data_enc_bow)
    result[alpha]['bert']=dict(random={}, trained={})
    result[alpha]['bert24']=dict(random={}, trained={})
    result[alpha]['infersent'] = dict(random={}, trained={})

    for mode in ['random', 'trained']:
        for step in ['first', 'last']:
            result[alpha]['bert'][mode][step] = {}
            result[alpha]['bert24'][mode][step] = {}
            for layer in range(12):
                logging.info("Computing RSA/RSA_regress scores for {} {} {}".format(mode, step, layer))
                data_enc = dict(test=data['bert']['test'][mode][layer][step], ref=data['bert']['ref'][mode][layer][step])
                result[alpha]['bert'][mode][step][layer] = RSA_report(data_tk, data_enc)
            for layer in range(24):
                logging.info("Computing RSA/RSA_regress scores for {} {} {}".format(mode, step, layer))
                data_enc = dict(test=data['bert24']['test'][mode][layer][step], ref=data['bert24']['ref'][mode][layer][step])
                result[alpha]['bert24'][mode][step][layer] = RSA_report(data_tk, data_enc)

        result[alpha]['infersent'][mode] = RSA_report(data_tk, dict(test=data['infersent']['test'][mode], ref=data['infersent']['ref'][mode]))
    json.dump(result, open("report/RSA_natural.json", "w"), indent=2)

In [ ]:
# Code taken from: https://github.com/gchrupala/correlating-neural-and-symbolic-representations-of-language/blob/master/rsa/synsem.py

def ewt_embed():
    """Compute BoW, BERT and Infersent embeddings for the EWT data and save to file."""
    import rsa.pretrained as Pre
    from sklearn.feature_extraction.text import CountVectorizer
    def container():
        return dict(test=dict(random=dict(), trained=dict()),
                    ref=dict(random=dict(), trained=dict()))
    data = json.load(open("data/out/ewt.json"))
    emb = dict(bow={}, bert=container(), bert24=container(), infersent=container())
    # BOW
    v = CountVectorizer(tokenizer=lambda x: x.split())
    sent_ref = [s['sent'] for s in data['ref'] ]
    sent_test = [s['sent'] for s in data['test'] ]
    v.fit( sent_ref + sent_test )
    emb['bow']['test'] = torch.tensor(v.transform(sent_test).toarray(), dtype=torch.float)
    emb['bow']['ref'] =  torch.tensor(v.transform(sent_ref).toarray(), dtype=torch.float)

    for split in ['test', 'ref']:
      sent = [ datum['sent'] for datum in data[split] ]
      for mode in ["random", "trained"]:
        if mode == "random":
            rep24 = list(Pre.encode_bert(sent, trained=False, large=True))
            rep = list(Pre.encode_bert(sent, trained=False))
            emb['infersent'][split][mode] = Pre.encode_infersent(sent, trained=False)
        else:
            rep24 = list(Pre.encode_bert(sent, trained=True, large=True))
            rep = list(Pre.encode_bert(sent, trained=True))
            emb['infersent'][split][mode] =  Pre.encode_infersent(sent, trained=True)

        pooled24 = torch.cat([ pooled for _, pooled in rep24 ])
        pooled = torch.cat([ pooled for _, pooled in rep ])
        emb['bert24'][split][mode]['pooled'] = pooled24
        emb['bert'][split][mode]['pooled'] = pooled
        for i in range(len(rep24[0][0])):
            emb['bert24'][split][mode][i] = {}
            emb['bert24'][split][mode][i]['summed'] = torch.cat([ layers[i].sum(dim=1) for layers, _ in rep24 ], dim=0)
            emb['bert24'][split][mode][i]['first']  = torch.cat([ layers[i][:,0,:] for layers, _ in rep24], dim=0)
            emb['bert24'][split][mode][i]['last']   = torch.cat([ layers[i][:,-1,:] for layers, _ in rep24], dim=0)

        for i in range(len(rep[0][0])):
            emb['bert'][split][mode][i] = {}
            emb['bert'][split][mode][i]['summed'] = torch.cat([ layers[i].sum(dim=1) for layers, _ in rep ], dim=0)
            emb['bert'][split][mode][i]['first']  = torch.cat([ layers[i][:,0,:] for layers, _ in rep], dim=0)
            emb['bert'][split][mode][i]['last']   = torch.cat([ layers[i][:,-1,:] for layers, _ in rep], dim=0)
    torch.save(emb, "data/out/ewt_embed.pt")